In [20]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

from tinkoff.invest import Client
from tinkoff.invest import schemas
from datetime import datetime

from tinvest_robot_perevalov.news_fetcher import _init_sentiment_analyzer
from tinvest_robot_perevalov.trader import _quotation_to_float
from tinvest_robot_perevalov._config import CLASS_CODE_SPB

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
def str_to_date(datetime_string):
    # 'Jun 1 2005  1:33PM'
    return datetime.strptime(datetime_string, '%b %d %Y %I:%M%p')

In [5]:
news = pd.read_csv("../data/us_equities_news_dataset.csv")
news['year'] = news.release_date.apply(lambda x: int(x.split('-')[0]))

## Backtest

In [6]:
def get_daily_candle_price_by_date(date, candles):
    for c in candles:
        if date.month == c.time.month and date.day == c.time.day:
            return (_quotation_to_float(c.open) + _quotation_to_float(c.close))/2
        elif c.time.month > date.month:
            return False
    return False

In [7]:
sa = _init_sentiment_analyzer()

2022-05-21 19:35:34,042 - tinvest_robot_perevalov.news_fetcher - INFO - Downloading labels...
2022-05-21 19:35:34,152 - tinvest_robot_perevalov.news_fetcher - INFO - Initializing sentiment analyzer...
2022-05-21 19:35:34.214316: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-21 19:35:34.247219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-21 19:35:34.247541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Super with Max-Q Design computeCapability: 7.5
coreClock: 1.08GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 327.88GiB/s
2022-05-21 19:35:34.247564: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic librar

In [11]:
backtesting_dir = os.path.join("..", "data", "backtesting")

In [12]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
tickers = news.ticker.value_counts().index.values[:100]

In [13]:
for YEAR in years:
    for TICKER in tickers:
        # prepare data
        share_news = news[news.ticker == TICKER]
        share_news_year = share_news[share_news.year == YEAR]
        share_news_year.release_date = pd.to_datetime(share_news_year[share_news_year.year == YEAR].release_date)
        share_news_year = share_news_year.sort_values(by=['release_date'])

        # get candles
        with Client(token=os.environ["TINVEST_TOKEN"]) as client:
            try:
                share = client.instruments.share_by(id_type=schemas.InstrumentIdType.INSTRUMENT_ID_TYPE_TICKER, class_code=CLASS_CODE_SPB, id=TICKER).instrument
                candles = client.market_data.get_candles(figi=share.figi, from_=str_to_date(f"Jan 1 {YEAR}  11:59PM"), to=str_to_date(f"Dec 31 {YEAR}  11:59PM"), interval=schemas.CandleInterval.CANDLE_INTERVAL_DAY)
            except Exception as e:
                print(str(e))
                continue
        
        if len(candles.candles) == 0:
            continue

        balance = _quotation_to_float(candles.candles[0].open)*1.5 # current balance is 150% from the first year's day opening prices of the share
        initial_balance = float(balance) # set up initial balance

        n_shares = 0 # we don't have any shares at the very beginning
        
        # iterate over each news article
        operation_type_lst, price_lst, balance_lst, date_lst = [], [], [], []
        for i, row in share_news_year.iterrows():
            if str(" " + TICKER + " ") not in row.title and share.name not in row.title: # double check if the news about the particular company
                continue
            sentiment = sa.predict_sentiment(row.title)
            if sentiment == 'positive': # buy
                price = get_daily_candle_price_by_date(row.release_date, candles.candles)
                if price and balance >= price:
                    balance -= price # we buy 1 lot
                    n_shares += 1 # we buy 1 lot
                    ## execute order ##
                    operation_type_lst.append("BUY")
                    price_lst.append(price)
                    balance_lst.append(balance)
                    date_lst.append(row.release_date)
                    ## execute order ##
            elif sentiment == 'negative': # sell
                price = get_daily_candle_price_by_date(row.release_date, candles.candles)
                if price and n_shares > 0:
                    balance += price # we sell 1 lot
                    n_shares -= 1 # we sell 1 lot
                    ## execute order ##
                    operation_type_lst.append("SELL")
                    price_lst.append(price)
                    balance_lst.append(balance)
                    date_lst.append(row.release_date)
                    ## execute order ##

        if n_shares > 0:
            last_years_price = _quotation_to_float(candles.candles[-1].close)
            balance += n_shares*last_years_price
            print(f"{YEAR}-{TICKER} INIT BALANCE: {initial_balance} FINAL BALANCE: {balance} + (SHARES {n_shares})")
        else:
            print(f"{YEAR}-{TICKER} INIT BALANCE: {initial_balance} FINAL BALANCE: {balance}")
        
        pd.DataFrame.from_dict({'operationType': operation_type_lst, 'price': price_lst, "balance": balance_lst, "date": date_lst}).to_csv(os.path.join(backtesting_dir, f"{YEAR}-{TICKER}-{initial_balance}-{balance}.csv"), index=False)

/home/ins-alex/.local/lib/python3.8/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
2022-05-21 19:35:59,986 - tinkoff.invest.logging - INFO - c43271c693fb5502a31ff82f8243c8eb ShareBy
2022-05-21 19:36:00,097 - tinkoff.invest.logging - INFO - c0fa5799ca307a6d7e3ca65c9b55095e GetCandles


2012-AAPL INIT BALANCE: 21.93 FINAL BALANCE: 22.01 + (SHARES 1)


2022-05-21 19:36:07,354 - tinkoff.invest.logging - INFO - 2d940ba1e35bfe91ea541ab4a357a2b6 ShareBy
2022-05-21 19:36:07,441 - tinkoff.invest.logging - INFO - 4e9e8333630704941dc339169153b602 GetCandles


2012-MSFT INIT BALANCE: 39.825 FINAL BALANCE: 39.825


2022-05-21 19:36:07,777 - tinkoff.invest.logging - INFO - fd2093da4aee5e4df96b667b0af0f5dd ShareBy
2022-05-21 19:36:07,893 - tinkoff.invest.logging - INFO - d938bfd760c25f79451b65e2ae846d76 GetCandles


...

2020-URBN INIT BALANCE: 41.730000000000004 FINAL BALANCE: 40.86000000000001 + (SHARES 1)


2022-05-21 20:06:54,622 - tinkoff.invest.logging - ERROR - d31725f72f4a4c99b3a6a562fae882cc ShareBy NOT_FOUND 50002


(<StatusCode.NOT_FOUND: (5, 'not found')>, '50002', Metadata(tracking_id='d31725f72f4a4c99b3a6a562fae882cc', ratelimit_limit='200, 200;w=60', ratelimit_remaining=145, ratelimit_reset=6, message='instrument not found'))


2022-05-21 20:06:54,854 - tinkoff.invest.logging - INFO - 4a05e7ac4d0408afab9eab7914c5cbda ShareBy
2022-05-21 20:06:54,958 - tinkoff.invest.logging - INFO - 7f8c8d21ea311cd547bcdf92cfe0dda1 GetCandles


2020-BMY INIT BALANCE: 97.35000000000001 FINAL BALANCE: 95.41 + (SHARES 1)


## Analyze backtest results

In [14]:
files = os.listdir(backtesting_dir)
year_lst, ticker_lst, init_balance_lst, final_balance_lst, income_lst, order_cnt_lst = [], [], [], [], [], []

for f in files: 
    params = f.replace(".csv", "").split("-")

    with open(os.path.join(backtesting_dir, f)) as file:
        order_cnt = len([l for l in file]) - 1 

    year_lst.append(int(params[0]))
    ticker_lst.append(params[1])
    init_balance_lst.append(float(params[2]))
    final_balance_lst.append(float(params[3]))
    income_lst.append(float(params[3])/float(params[2])*100 - 100)
    order_cnt_lst.append(order_cnt)

df = pd.DataFrame.from_dict({'year': year_lst, 'ticker': ticker_lst, 'initBalance': init_balance_lst, 'finalBalance': final_balance_lst, 'income': income_lst, 'orderCnt': order_cnt_lst})

In [15]:
df.describe()

,year,initBalance,finalBalance,income,orderCnt
count,787.000000,787.000000,787.000000,787.000000,787.000000
mean,2016.060991,170.044201,183.424737,5.810266,7.397713
std,2.596992,246.016782,291.454665,27.768408,23.625754
min,2012.000000,4.350900,4.710000,-46.860747,0.000000
25%,2014.000000,57.495000,57.417500,0.000000,0.000000
50%,2016.000000,97.800000,101.610000,0.000000,2.000000
75%,2018.000000,182.017500,193.870000,9.172210,6.000000
max,2020.000000,2794.110000,4186.760000,485.211936,235.000000


In [17]:
df.to_csv("../data/backtesting-stats.csv", index=False)

In [19]:
df.ticker.nunique()

90

In [22]:
df_year_mean = df.groupby("year")['income', 'orderCnt'].agg("mean").reset_index().sort_values(["year"])
fig = px.bar(df_year_mean, x='year', y='income', color='orderCnt', labels={'orderCnt': 'Ср. число операций по бумаге', 'year': 'Год', 'income': 'Средний доход, % (годовых)'}, text_auto=True, title="Средний доход по годам в процентах годовых по всем бумагам")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), font=dict(size=18), modebar_orientation='v')
fig.show()

pio.write_image(fig, '../data/resources/income-per-year.jpg', width=1200, height=600)

In [26]:
df_ticker_mean = df.groupby("ticker")['income', 'orderCnt'].agg("mean").reset_index().sort_values(["income"])
fig = px.bar(df_ticker_mean, x='ticker', y='income', color='orderCnt', labels={'orderCnt': 'Ср. число операций по бумаге', 'ticker': 'Тикер', 'income': 'Средний доход, % (годовых)'}, title="Средний доход по тикерам в процентах годовых по всем бумагам")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), font=dict(size=18), modebar_orientation='v')
fig.show()

pio.write_image(fig, '../data/resources/income-per-ticker.jpg', width=2000, height=600)

In [25]:
df_ticker_mean

,ticker,income,orderCnt
61,MYL,-7.301084,2.777778
74,SLB,-5.865283,3.777778
4,AGN,-2.261676,6.777778
28,CVX,-1.496270,9.777778
0,AA,-1.315257,1.000000
...,...,...,...
82,UBER,17.745172,59.500000
6,AMZN,17.916933,2.444444
1,AAPL,18.684469,128.888889
62,NFLX,28.621755,25.444444


In [ ]:
# TODO: - common db sqlite (created in the news fetcher), - news_fetcher and analyzer into db, - scheduled task for monitoring the news and making operations (save operations into db)
# volumes for db and models
# sandbox_mode = True/False, reset_sandbox = True/False, initial balance
# fetch/monitoring frequency
# model confidence threshold (for operations)
# ranked list of tickers to buy
# list of rss sources